# Darcy equation

In this tutorial we present how to solve an evolutionary Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Let $(0,T)$ with $10=T>0$ be the overall simulation period. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {- \rho g \nabla y}\\
p_t + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \times (0,T] \qquad p = \rho g \text{ on } \partial_{bottom} \Omega \times (0,T] \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \times (0,T] $$
and initial conditions:
$$ p|_{t=0} = (1-y) \rho g \text{ in } \Omega \qquad q|_{t=0} = 0 \text{ in } \Omega $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
%load_ext Cython

In [2]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
import scipy.integrate as integrate

import porepy as pp
import pygeon as pg

import sympy as sp

from math import ceil, floor, log10, exp

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


### Parameters

In [3]:
# Set the maximum number of iterations of the non-linear solver
K = 50

# L-scheme parameter
L = 3.501e-2 #0.1

# Set the mesh refinment
N = 40

# Set the number of steps (excluding the initial condition)
num_steps = 9

quad_order = 3

# Simulation time length
T = 9/48

# Time switch conditions (for the boundary condition)
dt_D = 3/48

# Fluid density
rho = 1000

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-10
rel_tol = 1e-5

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = 'hard_coupling'

In [4]:
def initial_h(x): return 1

In [5]:
# Van Genuchten model parameters ( relative permeability model )
theta_s = 0.396
theta_r = 0.131

alpha = 0.423

n = 2.06
K_s = 4.96e-2

m = 1 - 1/n

In [6]:
# Time step
dt   = (T-0)/num_steps

### $\theta$ and $K$

In [7]:
# Symbolic psi
psi_var = sp.Symbol('psi', negative=True)

# Symbolic Theta
theta_expression = theta_r + (theta_s - theta_r) / (1 + (-alpha * psi_var) ** n) ** m
effective_saturation = (theta_expression - theta_r) / (theta_s - theta_r)

# Symbolic Conductivity K
hydraulic_conductivity_expression = K_s * (effective_saturation ** 0.5) * ( 1 - (1 - effective_saturation ** (1 / m)) ** m ) ** 2

In [8]:
# Theta lambda
theta_lambda = sp.lambdify(psi_var, theta_expression, 'numpy')

# Conductivity tensor lambda
conductivity_lambda = sp.lambdify(psi_var, hydraulic_conductivity_expression, 'numpy')

In [9]:
# Actual (and final) theta function
def theta(psi):
    mask = np.where(psi < 0)
    res = np.ones_like(psi) * theta_s
    res[mask] = theta_lambda(psi[mask])

    return res

In [10]:
# Actual (and final) theta function
def conductivity(psi):
    if psi < 0:
        return conductivity_lambda(psi)
    else:
        return K_s

### Mesh

In [11]:
bottom_grid =   pp.StructuredTriangleGrid([2*N, N], [2, 1])
bottom_grid.compute_geometry()

In [12]:
top_grid    = pp.StructuredTriangleGrid([2*N, 2*N], [2, 1])
top_grid.compute_geometry()

In [13]:
boundary_grid, boundary_face_map, boundary_node_map  = pp.partition.extract_subgrid(bottom_grid, 
                                                                                    bottom_grid.face_centers[1, :] == 1, 
                                                                                    faces=True)

### Discretization

In [14]:
key = "flow"

In [15]:
top_field = pg.Lagrange1(key)
top_data = {}

top_dof =  top_field.ndof( top_grid )

In [16]:
bottom_field = pg.Lagrange1(key)
bottom_data = {}

bottom_dof = bottom_field.ndof( bottom_grid )

In [17]:
boundary_field = pg.Lagrange1(key)
boundary_data = {}

boundary_dof = boundary_field.ndof( boundary_grid )

### Restrictor Matrices

In [18]:
bottom_delete_boundary_dof = np.zeros( shape=(bottom_dof - boundary_dof, bottom_dof) )
bottom_delete_boundary_dof[np.arange(bottom_dof - boundary_dof), bottom_grid.nodes[1, :] < 1] = 1
bottom_delete_boundary_dof = sps.csr_matrix(bottom_delete_boundary_dof)

In [19]:
bottom_isolate_boundary_dof = np.zeros(shape=(boundary_dof, bottom_dof))
bottom_isolate_boundary_dof[np.arange(boundary_dof), bottom_grid.nodes[1, :] == 1] = 1
bottom_isolate_boundary_dof = sps.csr_matrix(bottom_isolate_boundary_dof)

In [20]:
top_delete_boundary_dof = np.zeros( shape=(top_dof - boundary_dof, top_dof) )
top_delete_boundary_dof[np.arange(top_dof - boundary_dof), top_grid.nodes[1, :] > 0] = 1
top_delete_boundary_dof = sps.csr_matrix(top_delete_boundary_dof)

In [21]:
top_isolate_boundary_dof = np.zeros(shape=(boundary_dof, top_dof))
top_isolate_boundary_dof[np.arange(boundary_dof), top_grid.nodes[1, :] == 0] = 1
top_isolate_boundary_dof = sps.csr_matrix(top_isolate_boundary_dof)

### BCs

In [22]:
bot_dirichlet_flag  = np.array(bottom_delete_boundary_dof @ np.logical_and( bottom_grid.nodes[0, :] == 2, bottom_grid.nodes[1, :] < 1 ), dtype=bool)
bot_dirichlet_value = np.array(bot_dirichlet_flag, dtype=float)

In [23]:
top_dirichlet_flag  = np.array(top_delete_boundary_dof @ np.logical_and( top_grid.nodes[1, :] == 1, top_grid.nodes[0, :] < 1 ), dtype=bool)
top_dirichlet_value = lambda t: np.array(top_dirichlet_flag, dtype=float) * min( 3.2, 1 + 2.2 * t / dt_D )

In [24]:
dirichlet_flag  = np.hstack(( bot_dirichlet_flag, np.zeros(shape=boundary_dof, dtype=bool), top_dirichlet_flag ))
dirichlet_value = lambda t: np.hstack(( bot_dirichlet_value, np.zeros(shape=boundary_dof), top_dirichlet_value(t)))

### Matrix Assembly

##### Projection matrices

In [25]:
top_proj    = top_field.eval_at_cell_centers( top_grid )
bottom_proj = bottom_field.eval_at_cell_centers( bottom_grid )

##### Stifness

In [26]:
%%cython
import numpy as np

def find_ordering(coord: np.array):
    lx = np.argmin(coord[0, :])
    rx = np.argmax(coord[0, :])
    mx = np.setdiff1d(np.array([0,1,2]), np.array([lx, rx]))[0]

    # Vertical Alignment
    if np.abs( coord[0, lx] - coord[0, mx] ) < 1e-7:
        # lx and mx vertical aligned, rx no
        up =   lx if np.argmax(coord[1, np.array([lx, mx])]) == 0 else mx
        down = lx if np.argmin(coord[1, np.array([lx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, rx] ) < 1e-7:
            return [up, down, rx]
        else:
            return [down, rx, up]
    else:
        # rx and mx vertical aligned, lx no
        up =   rx if np.argmax(coord[1, np.array([rx, mx])]) == 0 else mx
        down = rx if np.argmin(coord[1, np.array([rx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, lx] ) < 1e-7:
            return [up, lx, down]
        else:
            return [down, up, lx]

In [27]:
def darcy_local_A(coord):

    ordering = find_ordering(coord)

    x0 = coord[:, ordering][:, 0]
    x1 = coord[:, ordering][:, 1]
    x2 = coord[:, ordering][:, 2]
    
    J_T_1_T = np.array([[x2[1]-x0[1], x0[1]-x1[1]],
                        [x0[0]-x2[0], x1[0]-x0[0]]]) / ((x1[0]-x0[0]) * (x2[1]-x0[1]) - (x2[0]-x0[0]) * (x1[1]-x0[1]))
    

    q_funcs = [J_T_1_T @ np.array([-1, -1]), J_T_1_T @ np.array([ 1, 0]), J_T_1_T @ np.array([0,  1])]

    M = np.zeros(shape=(3,3))

    jacobian = 1 / np.linalg.det( J_T_1_T.T )

    for i in range(3):
        for j in range(3):
            M[ ordering[i], ordering[j] ] = q_funcs[j].T @ np.array([[1, 0], [0, 1]]) @ q_funcs[i] * jacobian * K_s / 2

    return M

In [28]:
def richards_local_A(coord, psi):
    ordering = find_ordering(coord)

    x0 = coord[:, ordering][:, 0]
    x1 = coord[:, ordering][:, 1]
    x2 = coord[:, ordering][:, 2]
    
    J_T_1_T = np.array([[x2[1]-x0[1], x0[1]-x1[1]],
                        [x0[0]-x2[0], x1[0]-x0[0]]]) / ((x1[0]-x0[0]) * (x2[1]-x0[1]) - (x2[0]-x0[0]) * (x1[1]-x0[1]))
    

    q_funcs = [J_T_1_T @ np.array([-1, -1]), J_T_1_T @ np.array([ 1, 0]), J_T_1_T @ np.array([0,  1])]

    M = np.zeros(shape=(3,3))

    jacobian = 1 / np.linalg.det( J_T_1_T.T )
    ordered_psi = psi[ordering]

    psi_fun = lambda x,y: ordered_psi[0] + (ordered_psi[1] - ordered_psi[0]) * x + (ordered_psi[2] - ordered_psi[0]) * y

    for i in range(3):
        for j in range(3):
            integrand = lambda ys,x: np.array([conductivity(psi_fun(x,y)) for y in np.array(ys)])
            inside = lambda xs, n: np.array([integrate.fixed_quad(integrand, 0, 1-x, args=(x,), n=n)[0] for x in np.array(xs)])
            tmp = integrate.fixed_quad(inside, 0, 1, n=quad_order, args=(quad_order,))[0]

            M[ ordering[i], ordering[j] ] = tmp * q_funcs[j].T @ np.array([[2, 0], [0, 1/2]]) @ q_funcs[i] * jacobian

    return M

In [29]:
darcy_local_A(np.array([[0, 1, 0], [0, 0, 1]]))

array([[ 0.0496, -0.0248, -0.0248],
       [-0.0248,  0.0248,  0.    ],
       [-0.0248,  0.    ,  0.0248]])

In [30]:
richards_local_A(np.array([[0, 1, 0], [0, 0, 1]]), np.array([1,1,1]))

array([[ 0.062 , -0.0496, -0.0124],
       [-0.0496,  0.0496,  0.    ],
       [-0.0124,  0.    ,  0.0124]])

##### Mass

In [31]:
def local_mass(coord):
    ordering = find_ordering(coord)

    x0 = coord[:, ordering][:, 0]
    x1 = coord[:, ordering][:, 1]
    x2 = coord[:, ordering][:, 2]

    qs = [(lambda x,y: 1-x-y), (lambda x,y: x), (lambda x,y: y)]
    
    J = np.array([[x1[0]-x0[0], x2[0]-x0[0]],
                  [x1[1]-x0[1], x2[1]-x0[1]]])
    
    jacobian = np.linalg.det(J)
    M = np.zeros(shape=(3,3))

    for i in range(3):
        for j in range(3):
            integrand = lambda ys,x: np.array([qs[j](x,y) * qs[i](x,y) for y in np.array(ys)])
            inside = lambda xs, n: np.array([integrate.fixed_quad(integrand, 0, 1-x, args=(x,), n=n)[0] for x in np.array(xs)])
            tmp = integrate.fixed_quad(inside, 0, 1, n=2, args=(2,))[0]

            M[ ordering[i], ordering[j] ] = tmp * jacobian

    return M

In [32]:
local_mass(np.array([[0, 1, 0], [0, 0, 1]]))

array([[0.08333333, 0.04166667, 0.04166667],
       [0.04166667, 0.08333333, 0.04166667],
       [0.04166667, 0.04166667, 0.08333333]])

In [33]:
def global_stifness(psi):
    
    size = np.power(bottom_grid.dim + 1, 2) * bottom_grid.num_cells + np.power(top_grid.dim + 1, 2) * top_grid.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    for subdomain, base in zip([bottom_grid, top_grid], [0, boundary_grid.num_nodes * N]):
        _, _, _, _, _, node_coords = pp.map_geometry.map_grid(subdomain)

        # Allocate the data to store matrix entries, that's the most efficient
        # way to create a sparse matrix.

        cell_nodes = subdomain.cell_nodes()

        for c in np.arange(subdomain.num_cells):
            # For the current cell retrieve its nodes
            loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

            nodes_loc = cell_nodes.indices[loc]
            coord_loc = node_coords[:, nodes_loc]

            # Compute the stiff-H1 local matrix

            if subdomain == bottom_grid:
                A = darcy_local_A(coord_loc)
            else:
                A = richards_local_A(coord_loc, psi[nodes_loc])

            # Save values for stiff-H1 local matrix in the global structure
            cols = base + np.tile(nodes_loc, (nodes_loc.size, 1))

            loc_idx = slice(idx, idx + cols.size)
            rows_I[loc_idx] = cols.T.ravel()
            cols_J[loc_idx] = cols.ravel()
            data_IJ[loc_idx] = A.ravel()
            idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

In [34]:
def global_mass():
    
    size = np.power(bottom_grid.dim + 1, 2) * bottom_grid.num_cells + np.power(top_grid.dim + 1, 2) * top_grid.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    for subdomain, base, mult in zip([bottom_grid, top_grid], [0, boundary_grid.num_nodes * N], [1,2]):
        _, _, _, _, _, node_coords = pp.map_geometry.map_grid(subdomain)

        # Allocate the data to store matrix entries, that's the most efficient
        # way to create a sparse matrix.

        cell_nodes = subdomain.cell_nodes()

        for c in np.arange(subdomain.num_cells):
            # For the current cell retrieve its nodes
            loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

            nodes_loc = cell_nodes.indices[loc]
            coord_loc = node_coords[:, nodes_loc]

            # Compute the stiff-H1 local matrix
            
            A = local_mass(coord_loc) * mult

            # Save values for stiff-H1 local matrix in the global structure
            cols = base + np.tile(nodes_loc, (nodes_loc.size, 1))

            loc_idx = slice(idx, idx + cols.size)
            rows_I[loc_idx] = cols.T.ravel()
            cols_J[loc_idx] = cols.ravel()
            data_IJ[loc_idx] = A.ravel()
            idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

### Solving stage

In [35]:
# assemble initial solution
initial_solution = np.zeros(top_dof + bottom_dof - boundary_dof)

In [36]:
bottom_mask = np.zeros_like(initial_solution, dtype=bool)
bottom_mask[ : (bottom_dof) ] = True

internal_bottom_mask = np.zeros_like(bottom_mask, dtype=bool)
internal_bottom_mask[ : (bottom_dof - boundary_dof)] = True

top_mask = np.zeros_like(initial_solution, dtype=bool)
top_mask[(bottom_dof - boundary_dof) : ] = True

internal_top_mask = np.zeros_like(bottom_mask, dtype=bool)
internal_top_mask[ bottom_dof : ] = True

boundary_mask = np.zeros_like(bottom_mask, dtype=bool)
boundary_mask[(bottom_dof - boundary_dof) : bottom_dof] = True

In [37]:
initial_solution[top_mask]    =    top_field.interpolate(   top_grid, lambda x: 1)
initial_solution[bottom_mask] = bottom_field.interpolate(bottom_grid, lambda x: 1)

In [38]:
top_saver    = pp.Exporter(top_grid,    "top_sol", folder_name=output_directory)
bottom_saver = pp.Exporter(bottom_grid, "bottom_sol", folder_name=output_directory)

def save_step(current_sol, step):
    ins = list()

    ins.append((top_grid, "cell_h", top_proj @ current_sol[top_mask]))
    ins.append((top_grid, "cell_p", top_proj @ (current_sol[top_mask] - 1 - top_grid.nodes[1, :] * 2)))
    top_saver.write_vtu(ins, time_step=step)

    ins = list()

    ins.append((bottom_grid, "cell_h", bottom_proj @ current_sol[bottom_mask]))
    ins.append((bottom_grid, "cell_p", bottom_proj @ (current_sol[bottom_mask] - bottom_grid.nodes[1, :])))
    bottom_saver.write_vtu(ins, time_step=step)

In [39]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [40]:
# solve the problem

sol = [initial_solution]

t = 0

save_step(sol[-1], 0)

In [41]:
csv_base = os.path.join(output_directory, 'csv')

if os.path.exists(csv_base):
    shutil.rmtree(csv_base)
    
os.mkdir(csv_base)

In [42]:
glob_mass = global_mass()
top_mass = glob_mass[top_mask, :][:, top_mask]
bottom_mass = glob_mass[bottom_mask, :][:, bottom_mask]

In [43]:
# Time Loop
for step in range(1, ceil(T/dt) + 1):
    current_time = step * dt
    print('Time ' + str(round(current_time, 5)))
    
    time_rhs = np.zeros_like(sol[-1])

    time_rhs[top_mask] += (top_mass @ theta(sol[-1][top_mask] - (1 + top_grid.nodes[1,:] * 2))) / dt

    prev = sol[-1]

    for k in range(K):
        rhs = time_rhs.copy()

        rhs[top_mask] += L * top_mass @ prev[top_mask] / dt
        rhs[top_mask] -= (top_mass @ theta(prev[top_mask] - (1 + top_grid.nodes[1,:] * 2))) / dt
        #rhs[internal_bottom_mask] += L / dt * bottom_delete_boundary_dof @ bottom_mass @ bottom_delete_boundary_dof.T @ prev[internal_bottom_mask]

        global_stif = global_stifness( prev[top_mask] - (1 + top_grid.nodes[1,:] * 2) )

        bottom_mat = global_stif[bottom_mask, :][:, bottom_mask] #+ L / dt * bottom_mass
        top_mat    = global_stif[top_mask, :][:, top_mask] + L / dt * top_mass
        mid_mat = global_stif[boundary_mask, :][:, boundary_mask] + L / dt * glob_mass[boundary_mask, :][:, boundary_mask]
        
        spp = sps.bmat([[    bottom_delete_boundary_dof @ bottom_mat @ bottom_delete_boundary_dof.T, bottom_delete_boundary_dof @ bottom_mat @ bottom_isolate_boundary_dof.T,                                                             None],
                        [ bottom_isolate_boundary_dof @ bottom_mat.T @ bottom_delete_boundary_dof.T,                                                                 mid_mat, top_isolate_boundary_dof @ top_mat.T @ top_delete_boundary_dof.T],
                        [                                                                      None,          top_delete_boundary_dof @ top_mat @ top_isolate_boundary_dof.T,    top_delete_boundary_dof @ top_mat @ top_delete_boundary_dof.T]], format = 'csc')
        
        ls = pg.LinearSystem(spp, rhs)
        ls.flag_ess_bc(dirichlet_flag, dirichlet_value(current_time))

        current = ls.solve()

        # Check if we have reached convergence
        rel_err_psi  = np.sqrt( (current - prev).T @ glob_mass @ (current - prev) )
        abs_err_prev = np.sqrt( prev.T @ glob_mass @ prev )

        # Log message with error and current iteration
        print('Iteration #' + format(k+1, '0' + str(ceil(log10(K)) + 1) + 'd') + ', error L2 relative psi: ' 
              + format(rel_err_psi, str(5 + ceil(log10(1 / abs_tol)) + 4) + '.' + str(ceil(log10(1 / abs_tol)) + 4) + 'f') )
        
        if rel_err_psi > abs_tol + rel_tol * abs_err_prev:
            prev = current.copy()
        else:
            break

    sol.append( current )

    save_step(sol[-1], step)
    print('')    

Time 0.02083
Iteration #001, error L2 relative psi:    0.12573597055050
Iteration #002, error L2 relative psi:    0.02079067102851
Iteration #003, error L2 relative psi:    0.00137023599990
Iteration #004, error L2 relative psi:    0.00015326008110
Iteration #005, error L2 relative psi:    0.00001044186161

Time 0.04167
Iteration #001, error L2 relative psi:    0.19529748926553
Iteration #002, error L2 relative psi:    0.05235463142435
Iteration #003, error L2 relative psi:    0.00957800772739
Iteration #004, error L2 relative psi:    0.00120371632954
Iteration #005, error L2 relative psi:    0.00039739159484
Iteration #006, error L2 relative psi:    0.00004729806294
Iteration #007, error L2 relative psi:    0.00001679188012

Time 0.0625
Iteration #001, error L2 relative psi:    0.27753939606979
Iteration #002, error L2 relative psi:    0.09115409920040
Iteration #003, error L2 relative psi:    0.02486223272082
Iteration #004, error L2 relative psi:    0.00432859333192
Iteration #005, 

bottom_mat = stifness(bottom_grid, None)

# Time Loop
for step in range(1, ceil(T/dt) + 1):
    current_time = step * dt
    print('Time ' + str(round(current_time, 5)))
    
    time_rhs = np.zeros_like(sol[-1])

    time_rhs[top_mask] += (top_mass @ theta(sol[-1][top_mask] - top_grid.nodes[1,:] - 1)) / dt

    prev = sol[-1]

    for k in range(K):
        rhs = time_rhs.copy()

        rhs[top_mask] += L * top_mass @ prev[top_mask] / dt
        rhs[top_mask] -= (top_mass @ theta(prev[top_mask] - top_grid.nodes[1,:] - 1)) / dt

        top_mat    = stifness(top_grid, prev[top_mask] - top_grid.nodes[1,:] - 1) + L / dt * top_mass
        mid_mat = top_isolate_boundary_dof @ top_mat @ top_isolate_boundary_dof.T + bottom_isolate_boundary_dof @ bottom_mat @ bottom_isolate_boundary_dof.T

        spp = sps.bmat([[    bottom_delete_boundary_dof @ bottom_mat @ bottom_delete_boundary_dof.T, bottom_delete_boundary_dof @ bottom_mat @ bottom_isolate_boundary_dof.T,                                                             None],
                        [ bottom_isolate_boundary_dof @ bottom_mat.T @ bottom_delete_boundary_dof.T,                                                                 mid_mat, top_isolate_boundary_dof @ top_mat.T @ top_delete_boundary_dof.T],
                        [                                                                      None,          top_delete_boundary_dof @ top_mat @ top_isolate_boundary_dof.T,    top_delete_boundary_dof @ top_mat @ top_delete_boundary_dof.T]], format = 'csc')
        
        ls = pg.LinearSystem(spp, rhs)
        ls.flag_ess_bc(dirichlet_flag, dirichlet_value(current_time))

        current = ls.solve()

        # Check if we have reached convergence
        rel_err_psi  = np.sqrt( (current - prev)[top_mask].T @ top_mass @ (current - prev)[top_mask] + (current - prev)[bottom_mask].T @ bottom_mass @ (current - prev)[bottom_mask] )
        abs_err_prev = np.sqrt( (prev)[top_mask].T @ top_mass @ (prev)[top_mask] + (prev)[bottom_mask].T @ bottom_mass @ (prev)[bottom_mask] )

        # Log message with error and current iteration
        print('Iteration #' + format(k+1, '0' + str(ceil(log10(K)) + 1) + 'd') + ', error L2 relative psi: ' 
              + format(rel_err_psi, str(5 + ceil(log10(1 / abs_tol)) + 4) + '.' + str(ceil(log10(1 / abs_tol)) + 4) + 'f') )
        
        if rel_err_psi > abs_tol + rel_tol * abs_err_prev:
            prev = current.copy()
        else:
            break

    sol.append( current )

    save_step(sol[-1], step)
    print('')    